In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=428ed61acdc0079692a666efffca1a436cd5d18bda97115b7f83755653899c7e
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [ ]:
#%% import
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
import re


#%% functions

def get_sitemap_links(sitemap_url):
    """
    Pobiera linki z sitemap.xml
    """
    try:
        r = requests.get(sitemap_url)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'xml')  # Użyj parsera XML!

        # W sitemap.xml linki są w tagach <loc>
        links = [loc.text.strip() for loc in soup.find_all('loc')]

        return links
    except Exception as e:
        print(f"Błąd pobierania sitemap: {e}")
        return []


def filter_article_links(all_links):
    """
    Filtruje linki - zostawia tylko artykuły
    WordPress.com struktura artykułów: /YYYY/MM/DD/slug/ lub /YYYY/MM/slug/
    """
    article_links = []
    excluded_count = {'sitemap': 0, 'category': 0, 'tag': 0, 'page': 0, 'author': 0, 'files': 0, 'other': 0}

    for link in all_links:
        # 0. Wykluczamy pliki (obrazy, PDF, etc.)
        file_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp', '.pdf', '.doc', '.docx', '.zip', '.mp4', '.mp3']
        if any(link.lower().endswith(ext) for ext in file_extensions):
            excluded_count['files'] += 1
            continue

        # Wykluczamy wp-content (uploads)
        if '/wp-content/' in link:
            excluded_count['files'] += 1
            continue

        # 1. Wykluczamy sitemapy
        if 'sitemap' in link.lower():
            excluded_count['sitemap'] += 1
            continue

        # 2. Wykluczamy kategorie i tagi
        if '/category/' in link or '/tag/' in link:
            excluded_count['category'] += 1
            continue

        # 3. Wykluczamy autorów
        if '/author/' in link:
            excluded_count['author'] += 1
            continue

        # 4. Wykluczamy strony paginacji
        if '/page/' in link:
            excluded_count['page'] += 1
            continue

        # 5. Wykluczamy archiwum dat (tylko /YYYY/MM/ bez dalszej ścieżki)
        # Wzorzec: kończy się na /YYYY/MM/ lub /YYYY/
        if re.match(r'.*\/\d{4}\/(0\d|1[0-2])\/?$', link):
            excluded_count['other'] += 1
            continue
        if re.match(r'.*\/\d{4}\/?$', link):
            excluded_count['other'] += 1
            continue

        # 6. AKCEPTUJEMY: artykuły z datą w ścieżce
        # WordPress zwykle: /YYYY/MM/DD/slug/ lub /YYYY/MM/slug/
        # Musi mieć coś PO dacie (slug artykułu)
        if re.search(r'\/\d{4}\/\d{2}\/.+\/', link):
            article_links.append(link)
            continue

        # 7. Wykluczamy główną stronę, about, contact, etc.
        # Jeśli URL jest bardzo krótki (tylko domena) lub statyczne strony
        path = link.replace('https://krytykat.wordpress.com', '').strip('/')
        if not path or path in ['about', 'contact', 'o-blogu', 'archiwum']:
            excluded_count['other'] += 1
            continue

        # 8. Jeśli nie pasuje do żadnej kategorii ale wygląda jak artykuł
        # (ma tytuł w URL, nie jest krótki)
        if len(path.split('/')) >= 2:  # Przynajmniej 2 segmenty
            article_links.append(link)

    # Raport filtrowania
    print(f"\nStatystyki filtrowania:")
    print(f"  Pliki (obrazy, PDF, etc.): {excluded_count['files']}")
    print(f"  Sitemapy: {excluded_count['sitemap']}")
    print(f"  Kategorie/tagi: {excluded_count['category']}")
    print(f"  Autorzy: {excluded_count['author']}")
    print(f"  Paginacja: {excluded_count['page']}")
    print(f"  Inne (archiwum, strony): {excluded_count['other']}")
    print(f"  ✅ ZAAKCEPTOWANO: {len(article_links)}")

    return article_links


def get_all_article_links(sitemap_url):
    """
    Główna funkcja - pobiera wszystkie linki z sitemap
    """
    print("Krok 1: Pobieranie sitemap...")
    all_links = get_sitemap_links(sitemap_url)

    print(f"Znaleziono {len(all_links)} linków w sitemap")

    print("\nKrok 2: Filtrowanie linków do artykułów...")
    article_links = filter_article_links(all_links)

    print(f"Po filtrowaniu: {len(article_links)} artykułów")

    # Pokaż przykłady
    if article_links:
        print("\nPrzykładowe linki (pierwsze 5):")
        for i, link in enumerate(article_links[:5], 1):
            print(f"  {i}. {link}")

    return article_links, []


#%% main execution

if __name__ == "__main__":
    sitemap_url = "https://krytykat.wordpress.com/sitemap.xml"

    print("="*60)
    print("Pobieranie linków z sitemap krytykat.wordpress.com")
    print("="*60)

    # Pobierz wszystkie linki
    article_links, errors = get_all_article_links(sitemap_url)

    if not article_links:
        print("\n⚠️  Nie znaleziono artykułów!")
        print("Sprawdź ręcznie sitemap:")
        print(f"  {sitemap_url}")
        exit(1)

    # Sortuj alfabetycznie
    article_links.sort()

    # Zapisz do pliku tekstowego
    with open('krytykat_linki.txt', 'w', encoding='utf-8') as f:
        for link in article_links:
            f.write(link + '\n')

    # Zapisz do JSON (z metadanymi)
    output_data = {
        'source': sitemap_url,
        'total_links': len(article_links),
        'links': article_links,
        'errors': errors
    }

    with open('krytykat_linki.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    # Raport
    print("\n" + "="*60)
    print("RAPORT")
    print("="*60)
    print(f"Znaleziono artykułów: {len(article_links)}")
    print(f"\nZapisano do:")
    print(f"  - krytykat_linki.txt")
    print(f"  - krytykat_linki.json")
    print("="*60)

In [ ]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        # Jeśli już jest w formacie YYYY-MM-DD
        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        # Jeśli jest datetime z czasem
        if 'T' in date_string:
            return date_string.split('T')[0]

        # Słownik z obiema formami miesięcy
        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        print(f"Błąd konwersji daty '{date_string}': {e}")
        return "no date"


def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu z krytykat.wordpress.com
    """
    try:
        r = requests.get(article_link)
        r.encoding = 'utf-8'
        html_text = r.text

        while '429 Too Many Requests' in html_text:
            time.sleep(5)
            r = requests.get(article_link)
            r.encoding = 'utf-8'
            html_text = r.text

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji
        try:
            # WordPress.com może mieć różne struktury
            date_element = soup.find('time')  # Bez klasy - szukamy dowolnego <time>
            if date_element:
                # Próbuj datetime attribute (najlepsze)
                date_text = date_element.get('datetime')
                if not date_text:
                    # Jeśli nie ma datetime, użyj tekstu
                    date_text = date_element.get_text(strip=True)
                date_of_publication = date_change_format(date_text)
            else:
                # Alternatywnie szukaj w spanach/divach z "date"
                date_element = soup.find(['span', 'div'], class_=lambda x: x and 'date' in str(x).lower())
                if date_element:
                    date_text = date_element.get_text(strip=True)
                    date_of_publication = date_change_format(date_text)
                else:
                    date_of_publication = "no date"
        except Exception as e:
            print(f"Błąd parsowania daty dla {article_link}: {e}")
            date_of_publication = "no date"

        # Tytuł
        try:
            title_element = soup.find('h1', class_='entry-title')
            if not title_element:
                title_element = soup.find('h1')
            title = title_element.get_text(strip=True) if title_element else "no title"
        except:
            title = "no title"

        # Autor
        try:
            author_element = soup.find('a', class_='url fn n')
            if not author_element:
                author_element = soup.find('span', class_='author')
            if not author_element:
                author_element = soup.find('a', rel='author')

            author = author_element.get_text(strip=True) if author_element else "no author"
        except:
            author = "no author"

        # Treść artykułu
        try:
            article_body = soup.find('div', class_='entry-content')
            if article_body:
                text = article_body.get_text(strip=True).replace('\n', ' ').replace('\xa0', ' ')
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria
        try:
            category_links = soup.find_all('a', rel='category tag')
            if not category_links:
                category_links = soup.find_all('a', rel='category')

            if category_links:
                categories = [cat.get_text(strip=True) for cat in category_links]
                category = ' | '.join(categories)
            else:
                category = "no category"
        except:
            category = "no category"

        # Tagi
        try:
            tag_links = soup.find_all('a', rel='tag')
            if tag_links:
                tags = [tag.get_text(strip=True) for tag in tag_links]
                tags_str = ' | '.join(tags)
            else:
                tags_str = None
        except:
            tags_str = None

        # Linki zewnętrzne
        try:
            if article_body:
                links = [a['href'] for a in article_body.find_all('a', href=True)]
                external_links = [link for link in links if not re.search(r'krytykat\.wordpress\.com|wordpress\.com', link)]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except (AttributeError, KeyError, IndexError):
            external_links = None

        # Zdjęcia
        try:
            images = []

            # 1. Thumbnail / post-thumbnail
            thumbnail_div = soup.find('div', class_=lambda x: x and 'post-thumbnail' in str(x).lower())
            if thumbnail_div:
                thumb_img = thumbnail_div.find('img', src=True)
                if thumb_img:
                    images.append(thumb_img['src'])

            # 2. Featured image
            if not images:
                featured_img = soup.find('img', class_=lambda x: x and 'wp-post-image' in str(x).lower())
                if featured_img and featured_img.get('src'):
                    images.append(featured_img['src'])

            # 3. Zdjęcia w treści
            if article_body:
                content_images = [img['src'] for img in article_body.find_all('img', src=True) if img.get('src')]
                for img_src in content_images:
                    if img_src not in images:
                        images.append(img_src)

            # 4. Inne miejsca
            for container_class in ['entry-header', 'post-header']:
                header = soup.find('div', class_=container_class)
                if header:
                    header_images = [img['src'] for img in header.find_all('img', src=True) if img.get('src')]
                    for img_src in header_images:
                        if img_src not in images:
                            images.append(img_src)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except (AttributeError, KeyError, IndexError) as e:
            has_images = False
            photos_links = None

        # Filmy (iframe)
        try:
            if article_body:
                iframes = [iframe['src'] for iframe in article_body.find_all('iframe', src=True)]
                has_videos = len(iframes) > 0
            else:
                has_videos = False
        except:
            has_videos = False

        dictionary_of_article = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Tagi": tags_str,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(dictionary_of_article)

    except AttributeError as e:
        errors.append(article_link)
        print(f"Błąd dla {article_link}: {e}")
    except Exception as e:
        errors.append(article_link)
        print(f"Nieoczekiwany błąd dla {article_link}: {e}")


#%% main execution

if __name__ == "__main__":
    # Wczytaj linki z pliku
    try:
        with open('krytykat_linki.txt', 'r', encoding='utf-8') as f:
            article_links = [line.strip() for line in f if line.strip()]
        print(f"Wczytano {len(article_links)} linków z pliku")
    except FileNotFoundError:
        print("Nie znaleziono pliku krytykat_linki.txt")
        print("Użyj najpierw get_krytykat_links.py aby pobrać linki!")
        print("\nLub podaj linki ręcznie:")
        article_links = []

    if not article_links:
        print("Brak linków do przetworzenia!")
        exit(1)

    all_results = []
    errors = []

    print("\n" + "="*60)
    print("Rozpoczynam scraping artykułów z krytykat.wordpress.com")
    print("="*60 + "\n")

    # Scraping z progress barem
    with ThreadPoolExecutor(max_workers=5) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # Zapisywanie wyników
    timestamp = datetime.today().date()

    # JSON
    with open(f'krytykat_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    # Excel
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(f"krytykat_{timestamp}.xlsx",
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)

    # Raport
    print(f"\n{'='*60}")
    print(f"Scraping zakończony!")
    print(f"Przetworzono artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        if len(errors) > 10:
            print(f"  ... i {len(errors) - 10} więcej")
    print(f"\nPliki wyjściowe:")
    print(f"  - krytykat_{timestamp}.json")
    print(f"  - krytykat_{timestamp}.xlsx")
    print(f"{'='*60}\n")